In [9]:
import redis
import yaml
import numpy as np
import random
import dask.array as da

In [4]:
with open('../settings.yaml') as f:
    sets = yaml.load(f, Loader=yaml.FullLoader)

In [5]:
r = redis.Redis(sets['redis']['host'], port=sets['redis']['port'], db=0)

In [18]:
#https://math.stackexchange.com/questions/2858815/understanding-formula-for-hyperplanes

In [6]:
H,W=100000,512
HPs = 1024 # number of hyperplanes, equals to number of bits in key

In [7]:
Y = da.random.normal(size=(H, W))

In [11]:
idxes = np.empty([HPs, 2], dtype=np.int)
for i in range(HPs):
    idxes[i,:]=np.random.choice(a=H, size=2, replace=False)

In [37]:
a=Y[idxes[:,0]]
b=Y[idxes[:,1]]

In [43]:
b_a=b-a
a_norm2=da.linalg.norm(a, axis=1)
b_norm2=da.linalg.norm(b, axis=1)
c=(b_norm2-a_norm2)/2

In [78]:
X = da.random.normal(size=(W))
#X=Y[idxes[10,0],:]

In [100]:
b_a

dask.array<sub, shape=(1024, 512), dtype=float64, chunksize=(3, 512), chunktype=numpy.ndarray>

In [86]:
q=da.ma.masked_greater(b_a.dot(X)-c,0)

In [87]:
qc=q.compute()

In [95]:
key=np.packbits(qc.mask)

In [109]:
keys=da.ma.masked_greater(Y.dot(b_a.T)-c,0)

In [112]:
keys.compute()

masked_array(
  data=[[--, -6.379714452280329, -17.408126407724005, ..., --, --,
         -30.786593438975665],
        [--, -10.73193769856973, --, ..., --, --, -38.396130081459845],
        [--, -56.89904153235883, -19.15664359042478, ..., --,
         -14.663298132865581, -33.43855375031051],
        ...,
        [-31.94095140246445, --, -15.969255978024337, ...,
         -21.137359634617027, --, -34.14087156665706],
        [-38.28440219200497, -16.780408604091228, --, ..., --,
         -9.132078984626327, -27.237102668824363],
        [-37.27659656647375, -13.252808561563414, -16.244853276603862,
         ..., --, --, -12.309011723700985]],
  mask=[[ True, False, False, ...,  True,  True, False],
        [ True, False,  True, ...,  True,  True, False],
        [ True, False, False, ...,  True, False, False],
        ...,
        [False,  True, False, ..., False,  True, False],
        [False, False,  True, ...,  True, False, False],
        [False, False, False, ...,  True,  True,

In [110]:
keys_bits=np.packbits(keys.mask)

AttributeError: 'Array' object has no attribute 'mask'

In [111]:
keys

dask.array<masked_greater, shape=(100000, 1024), dtype=float64, chunksize=(25000, 3), chunktype=numpy.MaskedArray>

In [21]:
a=np.random.rand(D)
a_norm2=np.dot(a,a)
b=np.random.rand(D)
b_norm2=np.dot(b,b)

In [22]:
for i in range(10):
    x=np.random.rand(D)
    print(np.dot((b-a),x)-0.5*(b_norm2-a_norm2))

-0.1710563516679867
0.09007390803510262
-0.3126568818384522
-0.6698706360217792
-0.37790388714794215
0.0135725719437913
0.06801615491527663
-0.4088421532486562
-0.17820569717080706
-0.5156129506644247
